# Elliptic Curve Cryptography
<https://en.wikipedia.org/wiki/Elliptic-curve_cryptography>

### Problems with RSA
 - RSA is based on the difficulty of factoring large integers
 - Quantum computers can factor large integers in polynomial time
 - Quantum computers can break RSA
 - Quantum computers can't break ECC
  - factorization is the trapdoor function for RSA, but it has never been proven that factorization is hard.
 

### Elliptic Curve
Let $E$ be an elliptic curve defined over a finite field $F_p$ where $p$ is a prime number. The set of points on $E$ is defined as:
$$E(F_p) = \{(x,y) \in F_p \times F_p | y^2 = x^3 + ax + b\} \cup \{\mathcal{O}\}$$
where $a,b \in F_p$ and $4a^3 + 27b^2 \neq 0$.

##### ....what?
#### Explain Like I'm Five
 - $E$ is a curve defined over a finite field $F_p$ where $p$ is a prime number
 - $E$ is the set of points $(x,y)$ that satisfy the equation $y^2 = x^3 + ax + b$
 - $a$ and $b$ are constants that define the curve
 - $4a^3 + 27b^2 \neq 0$ is a condition that ensures the curve is non-singular
 - $\mathcal{O}$ is the point at infinity

#### Point Addition
Let $P$ and $Q$ be points on $E$ and let $R$ be the point at infinity. The point addition operation is defined as:
$$P + Q = R$$
where $R$ is the point on $E$ that intersects the line between $P$ and $Q$.
#### Point Doubling
Let $P$ be a point on $E$ and let $R$ be the point at infinity. The point doubling operation is defined as:
$$P + P = R$$
where $R$ is the point on $E$ that intersects the tangent line of $P$.



#### Finding $X_3$ and $Y_3$
Let $P = (x_1, y_1)$ and $Q = (x_2, y_2)$ be points on $E$ and let $R = (x_3, y_3)$ be the point at infinity. The point addition operation is defined as:
$$x_3 = \lambda^2 - x_1 - x_2$$
$$y_3 = \lambda(x_1 - x_3) - y_1$$
where $\lambda$ is defined as:
$$\lambda = \frac{y_2 - y_1}{x_2 - x_1}$$


In [2]:
# A Simple implementation of Elliptic Curve Cryptography

# A Point on the Curve
class Point:
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f"({self.x}  -   {self.y})"
    
    
class SimpleEllipticCurve:
    def __init__(self, a: int, b: int):
        # y^2 = x^3 + ax + b
        # Bitcoin - a = 0, b = 7 (y^2 = x^3 + 7)
        self.a = a
        self.b = b
    
    # Naive Point Addition
    def point_addition(self, P: Point, Q: Point):
        x1, y1 = P.x, P.y
        x2, y2 = Q.x, Q.y
        
        if x1 == x2 and y1 == y2:
            # Point Doubling
            m = (3 * x1**2 + self.a) / (2 * y1)
        else:
            # Point Addition
            m = (y2 - y1) / (x2 - x1)
            
        x3 = m**2 - x1 - x2
        y3 = m * (x1 - x3) - y1
        
        return Point(x3, y3)
    
    # Double and Add Algorithm
    # O(m) where m is the number of bits in k
    def double_and_add(self, k: int, P: Point):
        # Convert k to binary
        k = bin(k)[3:]
        
        # Initialize R
        R = Point(P.x, P.y)
        
        # For each bit in k from left to right
        for bit in k:
            # R = 2R
            R = self.point_addition(R, R)
            
            # If the bit is 1, then R = R + P
            if bit == '1':
                R = self.point_addition(R, P)
                
        return R
    
    
ecc = SimpleEllipticCurve(0, 7)
P = Point(2, 4)
print(ecc.double_and_add(21, P))
    
    

(1.077486403391414  -   -3.0415352829993516)


# Double and Add Algorithm
In elliptic curve cryptography, the double and add algorithm is used to calculate $kP$ where $k$ is a scalar and $P$ is a point on the curve. The algorithm is as follows:
 - Convert $k$ to binary
 - Initialize $R = \mathcal{O}$
 - For each bit in $k$ from left to right:
  - $R = 2R$
  - If the bit is $1$, then $R = R + P$
 - Return $R$

In [3]:
# Elliptic Curve Cryptography Diffie-Hellman Key Exchange
import random

generator = Point(2, 4)

alice_random = random.randint(2, 1e4)
bob_random = random.randint(2, 1e4)

# public keys with double and add
alice_public_key = ecc.double_and_add(alice_random, generator)
bob_public_key = ecc.double_and_add(bob_random, generator)

# shared secret with double and add
alice_shared_secret = ecc.double_and_add(alice_random, bob_public_key)
bob_shared_secret = ecc.double_and_add(bob_random, alice_public_key)

print(alice_shared_secret)
print(bob_shared_secret)


(5.972713090756081  -   -14.8683017134625)
(5.9727189164285726  -   -14.868322679815325)
